In [ ]:
import pickle
import seaborn as sns
import imageio as io
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import tifffile
from tqdm.notebook import tqdm
import pathlib
import json
import glob
import PIL
import scanpy as sc
import napari

In [ ]:
output_folder = r'data\240719AnalysisDAM_TERM\fig_0925\Fig1F'

In [ ]:
adata_whole=sc.read(r'data\240719AnalysisDAM_TERM\adata_del2.h5ad')
adata_mg=sc.read(r'data\240719AnalysisDAM_TERM\adata_mg.h5ad')

In [ ]:
adata_app_wh=adata_whole[adata_whole.obs['batch']=='APP_1']
adata_te4_wh=adata_whole[adata_whole.obs['batch']=='TE4_3']
adata_wt_wh=adata_whole[adata_whole.obs['batch']=='WT_1']
adata_e4_wh=adata_whole[adata_whole.obs['batch']=='E4_2']

# adata_app_mg=adata[adata.obs['batch']=='APP_1']
# adata_te4_mg=adata[adata.obs['batch']=='TE4_3']
# adata_wt_mg=adata[adata.obs['batch']=='WT_1']
# adata_e4_mg=adata[adata.obs['batch']=='E4_2']

In [ ]:
cell_dict2={
'HPC': ['12','10','54'],
'CC': ['28'],
'VLMC':['43','32','61'],
'L2/3': ['17']   
}

adata_whole.obs['region_main_1'] = np.nan

for i in cell_dict2.keys():
    ind = pd.Series(adata_whole.obs['ddnres5']).isin(cell_dict2[i])
    adata_whole.obs.loc[ind,'region_main_1'] = i
adata_whole.obs['region_main_1'].fillna('Notclustered', inplace=True)

In [ ]:
adata=adata_whole.copy()

# Extended Fig2c

In [ ]:
cmap = ["#e6194B", "#3cb44b", "#ffe119", "#4363d8", "#f58231", "#911eb4", "#42d4f4", "#f032e6", "#bfef45",
        "#fabed4", "#469990", "#dcbeff", "#9A6324", "#fffac8", "#800000", "#aaffc3", "#808000", "#ffd8b1",
        "#000075", "#a9a9a9"]
def plot_cluster_scdata_str(scdata, cmap, clusters=[1, 2], transpose=1, flipx=1, flipy=1, tag='cluster', key='X_spatial'):
    import matplotlib.pyplot as plt
    import numpy as np
    
    # Generate unique color mappings for clusters
    unique_clusters = np.unique(scdata.obs[tag])
    cluster_to_color = {str(cluster): cmap[i % len(cmap)] for i, cluster in enumerate(unique_clusters)}
    
    x, y = (np.array(scdata.obsm[key]) * [flipx, flipy])[:, ::transpose].T
    plt.scatter(x, y, c='#E0E0E0', s=2, marker='.')#E0E0E0
    
    for cluster in clusters:
        cluster_ = str(cluster)
        inds = scdata.obs[tag] == cluster_
        x_ = x[inds]
        y_ = y[inds]
        col = cluster_to_color.get(cluster_, 'grey')  # Get the color from the mapping
        plt.scatter(x_, y_, c=col, s=4, marker='.', label=cluster_)
    
    plt.grid(False)
    plt.axis("off")
    plt.axis("equal")
    #plt.legend()
    plt.tight_layout()
    return plt.gcf()

In [ ]:
fig = plt.figure(figsize=(6, 6), facecolor="white")
fig = plot_cluster_scdata_str(adata_wt_wh, cmap, clusters=['HPC','CC','VLMC','L2/3'], transpose=1, flipx=-1, flipy=1, tag='region_main_1', key='X_multi_spatial')
plt.savefig(f'{output_folder}\\WT_4outline_1.tif', format='tif', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
fig = plt.figure(figsize=(6, 6), facecolor="white")
fig = plot_cluster_scdata_str(adata_e4_wh, cmap, clusters=['HPC','CC','VLMC','L2/3'], transpose=1, flipx=1, flipy=1, tag='region_main_1', key='X_multi_spatial')
plt.savefig(f'{output_folder}\\E4_4outline_1.tif', format='tif', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
fig = plt.figure(figsize=(6, 6), facecolor="white")
fig = plot_cluster_scdata_str(adata_app_wh, cmap, clusters=['HPC','CC','VLMC','L2/3'], transpose=1, flipx=1, flipy=-1, tag='region_main_1', key='X_multi_spatial')
plt.savefig(f'{output_folder}\\APP_4outline_1.tif', format='tif', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
fig = plt.figure(figsize=(6, 6), facecolor="white")
fig = plot_cluster_scdata_str(adata_te4_wh, cmap, clusters=['HPC','CC','VLMC','L2/3'], transpose=1, flipx=-1, flipy=1, tag='region_main_1', key='X_multi_spatial')
plt.savefig(f'{output_folder}\\TE4_4outline_1.tif', format='tif', dpi=300, bbox_inches='tight')
plt.show()

# Extended Fig2d

In [ ]:
X = np.array(adata.obsm['X_multi_spatial'])
res1_values = adata.obs['region_main_1'].values

cluster_colors = {
    'HPC': [1, 0, 0, 1],    # Red
    'CC': [0, 1, 0, 1],     # Green
    'VLMC': [0, 0, 1, 1],   # Blue
    'L2/3': [1, 1, 0, 1]    # Yellow
}

colors = np.zeros((X.shape[0], 4))  # Initialize with zeros for RGBA

for cluster, color in cluster_colors.items():
    colors[res1_values == cluster] = color

default_color = [0.5, 0.5, 0.5, 1] 
colors[~np.isin(res1_values, list(cluster_colors.keys()))] = default_color

scale_factor = 0.108  

viewer = napari.Viewer()
points_layer = viewer.add_points(
    X,
    size=30,
    face_color=colors,
    scale=[scale_factor, scale_factor],
    metadata={'unit': 'µm'}
)

viewer.scale_bar.visible = True
viewer.scale_bar.unit = 'µm'
viewer.scale_bar.position = 'bottom_right'
viewer.scale_bar.color = 'white'
viewer.scale_bar.font_size = 10

def adjust_point_size(event):
    zoom_level = event.value
    base_size = 6  
    new_size = base_size / zoom_level
    points_layer.size = new_size

viewer.camera.events.zoom.connect(adjust_point_size)

napari.run()


In [ ]:
points = viewer.layers[-1].data
print([[list(p) for p in lines]for lines in points])

In [ ]:
points_all=[[[22341.665483057783, 13443.375571635424], [19398.144068165067, 15165.057908648143], [16954.465912405078, 18219.655603348132], [15121.707295585085, 21551.9439975663], [14010.944497512364, 24773.156111977198], [13733.253797994183, 28827.440324942632], [15066.169155681453, 33548.1822167517], [19120.45336864689, 39101.99620711532], [23008.123161901414, 42212.13204171894], [25507.339457565045, 43933.81437873166], [31838.68740657956, 45655.49671574438], [34171.28928253228, 47099.48835323892], [36892.658137810446, 48765.632550348004], [40502.637231546796, 50265.16232774618], [42557.54840798133, 51264.848846011635], [43779.38748586133, 50043.00976813164], [42335.395848366796, 45377.806016226205], [40835.86607096861, 41379.05994316439], [42279.857708463154, 40657.064124417135], [44501.3833046086, 41434.598083068035], [45334.45540316314, 42934.12786046621], [46945.06146036859, 45433.34415612984], [48222.438678152226, 47377.17905275711], [50110.735434875845, 49376.552089288], [52332.26103102129, 50987.15814649345], [53720.714528612196, 52320.07350418071], [55275.782445914, 54374.98468061526], [54831.47732668492, 55930.05259791707], [51943.49405169584, 57151.89167579706], [47833.67169882677, 57762.811214737056], [43335.08236663225, 57873.88749454433], [37836.80651617226, 56540.97213685706], [34393.44184214682, 55374.67119888071], [30172.543209470477, 54208.37026090435], [25729.49201717959, 51931.30652485527], [21675.207804214147, 49598.704648902545], [17176.61847201962, 46433.03067439528], [13677.715658090547, 44044.89065853893], [9901.12214464329, 40434.91156480258], [6180.0667710996695, 35658.63153308987], [3903.0030350505876, 29993.741262918993], [4014.0793148578605, 26828.06728841173], [4958.227693219675, 23051.473774964474], [6957.600729750576, 21440.86771775903], [10289.889123968742, 19441.494681228127], [14566.325896548726, 14443.062089900875], [19231.529648454158, 13443.375571635424], [22175.051063346873, 13387.837431731788]], [[18458.38080582207, 68823.66927269976], [20990.48070528721, 68486.05595277106], [26983.11713402138, 69498.89591255713], [32047.31693295166, 71271.36584218271], [37871.14670172149, 72959.43244182614], [42850.943170669605, 75069.51569138042], [47324.319659724686, 76841.98562100602], [51206.872838904565, 79036.47220054247], [53738.972738369695, 81146.55545009676], [55933.459317906156, 84353.88198941927], [54667.40936817359, 89755.69510827823], [53485.762748423185, 92541.00499768989], [53485.762748423185, 94735.49157722633], [55595.84599797747, 95579.52487704805], [59056.3825272465, 96339.1548468876], [61419.67576674729, 95832.73486699458], [62010.499076622495, 93891.45827740463], [61250.86910678295, 84353.88198941927], [61250.86910678295, 82328.20206984715], [58381.155887389126, 81821.78208995414], [58634.36587733565, 78108.03557073859], [58634.36587733565, 73634.65908168351], [52810.53610856583, 68064.03930286021], [39896.82662129361, 65278.72941344855], [34241.80351248812, 62999.83950392993], [26223.48716418184, 62155.806204108216], [21581.30401516242, 61818.19288417953], [16939.12086614299, 62071.40287412604], [13140.971016945274, 62240.20953409038], [10018.047807604933, 63253.04949387644], [7063.931258228936, 65363.132743430724], [5460.267988567679, 66966.79601309197], [7317.14124817545, 68739.26594271758], [12212.53438714139, 69498.89591255713], [16263.894226285616, 72537.4157919153], [19893.237415518983, 75829.14566121997], [21074.88403526938, 75660.33900125562], [18205.170815875557, 71186.96251220055], [18373.9774758399, 68992.47593266409]], [[24197.807244609732, 125964.7268903613], [20484.060725394196, 126555.5502002365], [17107.927526107338, 128074.8101399156], [14153.810976731342, 129762.87673955903], [11452.904417301856, 130860.12002932724], [9511.627827711916, 133139.00993884588], [8329.981207961517, 136093.12648822187], [7654.754568104146, 138456.41972772268], [7739.157898086317, 143014.19954675992], [10186.854467569286, 147487.576035815], [12465.744377087913, 150863.70923510185], [15673.070916410426, 153395.809134567], [19555.624095590312, 156096.7156939965], [23016.160624859334, 158460.0089334973], [25801.47051427099, 159894.86554319417], [28586.78040368265, 162426.96544265933], [26307.890494164018, 165634.29198198183], [19133.60744567945, 166478.32528180355], [12887.761026998769, 163608.61206240972], [6557.511278335916, 159557.2522232655], [2168.538119263006, 154746.26241428175], [-616.771770148649, 142845.39288679557], [1240.1014894591199, 137612.38642790096], [2506.1514391916917, 132294.97663902416], [5966.6879684607175, 127315.18017007604], [10018.047807604944, 124107.85363075354], [15757.474246392598, 121997.77038119924], [19133.60744567945, 119887.68713164497], [22003.32066507328, 119296.86382176976], [25041.84054443145, 120900.52709143102], [27827.150433843104, 122757.4003510388], [30443.653663290417, 124783.08027061091], [31287.68696311213, 126217.93688030781], [31034.476973165612, 126386.74354027215]], [[98380.24938382162, 10125.544934185757], [104423.32070869772, 12074.922780919998], [108517.0141868396, 15388.865120368186], [113000.58323432833, 19677.496383183487], [114755.02329638913, 25330.692138712748], [115144.89886573597, 31763.639032935705], [111830.95652628779, 36247.20808042444], [105203.07184739142, 41120.65269725998], [97795.43602980135, 45019.408390728466], [90777.67578155812, 47553.59959148296], [85124.48002602886, 48528.28851485006], [81810.53768658068, 49113.10186887032], [80056.09762451987, 47163.72402213609], [75962.40414637799, 46383.97288344243], [73038.33737627664, 49113.10186887032], [70699.08396019557, 52232.106423645106], [70894.02174486898, 54766.297624399594], [74987.71522301088, 58080.23996384779], [82590.28882527437, 58080.23996384779], [89023.23571949732, 56910.61325580726], [96430.87153708738, 54766.297624399594], [103838.50735467745, 51062.479715604575], [111441.08095694095, 47163.72402213609], [118068.96563583732, 43070.03054399421], [123332.28582201975, 37611.7725731384], [124696.8503147337, 31763.639032935705], [124696.8503147337, 22601.563153284824], [121382.9079752855, 15973.67847438845], [116119.5877891031, 13634.425058307375], [110856.26760292068, 10710.358288206036], [105203.07184739142, 6616.664810064153], [100329.62723055584, 6421.727025390721], [97795.43602980135, 9345.793795492076]], [[109686.64089488014, 67242.31584349865], [105008.13406271799, 67827.12919751895], [98185.31159914819, 70361.32039827344], [92532.11584361893, 71920.82267566079], [87658.67122678335, 74455.01387641532], [84734.60445668202, 77379.08064651664], [83175.10217929464, 81862.64969400539], [80251.03540919328, 87515.84544953464], [80056.09762451987, 91024.72557365625], [82590.28882527437, 96677.9213291855], [84929.54224135545, 99796.92588396028], [88633.36015015046, 101161.49037667425], [91947.30248959867, 101551.36594602108], [96040.99596774054, 101746.30373069452], [99160.00052251531, 101746.30373069452], [102279.00507729006, 106229.87277818323], [102863.81843131034, 112467.88188773277], [102084.06729261664, 117536.26428924175], [96430.87153708738, 118316.01542793545], [88438.42236547705, 114027.38416512017], [78886.47091647935, 106229.87277818323], [71673.77288356268, 97652.61025255262], [69139.58168280819, 85761.40538747385], [72648.4618069298, 77184.14286184324], [77716.8442084388, 70751.19596762027], [86489.04451874283, 64708.12464274416], [98575.18716849505, 62953.684580683366], [102279.00507729006, 61004.30673394914], [108517.0141868396, 61004.30673394914], [113585.39658834861, 61784.057872642836], [115339.8366504094, 66462.56470480499], [112610.70766498148, 68022.06698219235]], [[85124.48002602886, 136250.29161789035], [82200.41325592753, 133131.2870631156], [82785.22660994779, 129427.46915432057], [85124.48002602886, 127867.96687693318], [89802.98685819101, 128842.65580030027], [92921.99141296578, 130597.0958623611], [96625.8093217608, 132156.5981397485], [101889.12950794323, 134495.85155582955], [105592.94741673827, 137030.04275658404], [108517.0141868396, 139759.17174201197], [110076.51646422698, 144632.61635884753], [110076.51646422698, 150480.74989905022], [106957.51190945222, 155159.25673121237], [103643.56957000402, 159447.8879940277], [99160.00052251531, 161787.14141010874], [95651.12039839369, 164126.39482618985], [92727.05362829236, 164906.14596488353], [95456.18261372027, 167440.33716563802], [102084.06729261664, 168999.83944302538], [108322.07640216617, 167440.33716563802], [111636.01874161437, 161397.26584076192], [115534.77443508283, 156913.6967932732], [118458.84120518417, 152430.12774578444], [119043.65455920443, 146776.9319902552], [117094.27671247021, 141903.48737341963], [113780.33437302202, 135275.60269452326], [101499.25393859639, 127478.0913075863], [93701.74255165947, 123969.21118346472], [87073.8578727631, 123189.46004477103], [83175.10217929464, 123189.46004477103], [80056.09762451987, 124164.14896813812], [78106.71977778563, 126113.52681487238], [77326.96863909194, 127867.96687693318], [78886.47091647935, 133521.16263246242], [80251.03540919328, 136055.35383321694], [82005.47547125409, 137030.04275658404]], [[160927.69236568073, 46103.96230628794], [164391.0038656591, 42897.1923989006], [167341.2321804554, 41101.401250763694], [170804.54368043374, 38792.52691744481], [174780.93836559405, 35970.56939894396], [178372.52066186786, 32763.79949155662], [180809.66579148226, 30711.46675082872], [183246.81092109665, 28402.59241750984], [181194.47818036875, 23143.4897693946], [182092.37375443717, 16601.679158324438], [180937.93658777772, 12112.201287982156], [178500.79145816332, 9546.78536207229], [175294.02155077603, 6083.473862093965], [171830.7100507977, 2491.8915658201463], [170804.54368043374, 183.01723250126375], [174396.12597670755, 567.8296213877435], [178885.60384704985, 2620.1623621156396], [184657.78968034705, 7494.452621344391], [187736.28879143888, 14164.534028710057], [189403.80914328032, 17884.387121279367], [190173.43392105328, 23784.843750872067], [187223.2056062569, 33789.965861920566], [181451.0197729597, 39562.15169521778], [173754.7719952301, 43153.73399149159], [166956.41979156894, 45590.879121105965], [162466.94192122665, 46488.77469517442]], [[183246.81092109665, 108828.38169478429], [179655.2286248228, 107673.94452812483], [175294.02155077603, 105621.61178739693], [169521.8357174788, 101645.21710223664], [163877.92068047708, 98181.9056022583], [159131.90121754384, 94718.59410228], [157207.83927311143, 91255.28260230167], [154385.88175461057, 86637.5339356639], [153616.2569768376, 82148.05606532162], [153103.17379165563, 78684.7445653433], [152846.63219906465, 74451.80828759201], [151307.38264351874, 72656.0171394551], [147459.25875465394, 72784.28793575059], [144509.03043985757, 79069.55695422979], [143226.32247690266, 84328.65960234503], [142841.51008801616, 87150.61712084588], [144380.75964356208, 90870.47021341517], [148228.8835324269, 96514.3852504169], [155540.31892127002, 103312.73745407806], [161440.77555086272, 107930.48612071582], [168495.66934711486, 111778.61000958062], [178115.9790692769, 116139.81708362742], [186838.39321737044, 118063.8790280598], [191199.6002914172, 117550.79584287782], [195945.61975435048, 116524.62947251389], [197741.41090248738, 114344.02593549048], [190943.05869882624, 112291.69319476259], [187223.2056062569, 109726.27726885272], [182990.26932850562, 106519.50736146538], [182348.91534702817, 107802.21532442034]], [[188249.3719766227, 173989.9454789236], [181194.47818037056, 172963.77910855965], [175550.56314336878, 170783.17557153627], [170804.54368043554, 168730.84283080837], [163493.10829159236, 166678.51009008047], [158362.2764397726, 163600.01097898863], [155925.1313101582, 161932.4906271472], [152846.63219906634, 158340.90833087338], [150922.57025463387, 154492.7844420086], [150409.48706945186, 150259.8481642573], [150409.4870694518, 146668.2658679835], [151948.73662499772, 142050.5172013457], [151948.7366249977, 140126.4552569133], [147844.07114354192, 143204.95436800516], [143354.59327319966, 147437.89064575644], [141430.53132876733, 150644.66055314377], [141302.26053247185, 153338.34727534914], [143226.32247690432, 160136.6994790103], [148357.15432872414, 166935.05168267144], [157720.92245829522, 171167.98796042276], [165032.3578471384, 175529.19503446954], [171574.16845820856, 177068.44459001545], [176448.45871743734, 178992.50653444786], [187094.93480996328, 178607.69414556137], [192482.30825437405, 177709.79857149292], [196843.51532842082, 177196.71538631094], [198511.0356802622, 176940.17379371997], [198511.0356802622, 174374.75786781005], [194534.64099510186, 171681.0711456047], [191199.60029141905, 171039.71716412724], [190301.70471735057, 168217.7596456264], [187351.47640255417, 166293.69770119397], [186710.12242107673, 170270.09238635428]], [[257515.6019761876, 47899.752475796355], [251102.06216141296, 48541.10645727382], [244175.4391614563, 48284.564864682834], [237890.1701429771, 46745.31530913691], [232374.52590227086, 43282.00380915858], [228269.8604208151, 39818.69230918025], [224678.27812454122, 36483.65160549742], [222112.86219863134, 31994.173735155146], [220958.42503197188, 28402.591438881325], [220573.6126430854, 24811.009142607505], [220060.52945790338, 21091.156050038193], [220958.42503197186, 16986.4905685824], [222112.86219863128, 14549.345438968026], [222369.40379122223, 12881.82508712661], [218521.27990235746, 14805.887031559012], [215571.05158756112, 18397.46932783283], [214801.42680978816, 19038.823309310297], [212749.0940690603, 22117.322420402146], [211722.92769869635, 25837.17551297146], [212749.09406906032, 31865.902938859654], [213518.71884683328, 37125.00558697489], [217751.6551245846, 42768.920623976606], [223267.29936529082, 47130.12769802339], [228526.4020134061, 50336.897605410726], [234170.31705040782, 51747.87636466116], [239301.14890222755, 54056.75069798004], [244945.06393922927, 54313.29229057102], [251615.14534659492, 53158.855123911584], [255206.72764286876, 51619.60556836567], [259567.93471691554, 52260.95954984313], [262774.70462430286, 52132.68875354764], [263416.05860578036, 50849.9807905927], [262518.1630317119, 50080.35601281974]], [[229809.10997636127, 109341.46390133775], [225704.44449490545, 110239.3594754062], [227756.77723563334, 113317.85858649806], [232887.6090874531, 112933.04619761158], [241481.75243925117, 110239.3594754062], [247510.47986513935, 106519.5063828369], [253667.47808732305, 103441.00727174504], [258285.2267539608, 99721.15417917575], [261363.7258650526, 95488.21790142445], [263929.1417909624, 89716.03206812723], [265853.20373539487, 84970.01260519399], [265853.2037353948, 80993.61792003368], [265083.5789576218, 77915.11880894184], [264570.49577243987, 75862.78606821394], [266366.2869205768, 75734.51527191846], [268803.43205019116, 77530.30642005536], [270984.0355872146, 85739.63738296695], [270855.76479091914, 92024.90640144613], [269701.3276242597, 98438.44621622081], [264057.412587258, 103184.46567915406], [259696.20551321126, 106391.23558654141], [253539.20729102756, 110752.44266058817], [245586.417920707, 114215.75416056652], [237377.0869577954, 116909.44088277187], [228526.40201340636, 118448.69043831779], [221856.32060604068, 118705.23203090878], [214288.34362460658, 118063.87804943131], [213133.90645794707, 116268.0869012944], [214544.88521719753, 113574.40017908905], [225447.9029023145, 109213.19310504226], [228911.2144022928, 108571.83912356479]], [[224550.00732824628, 172578.96647501606], [227756.77723563358, 175657.4655861079], [235324.7542170677, 176811.90275276735], [242764.46040220634, 175657.4655861079], [245329.8763281162, 175272.65319722143], [249691.083402163, 174374.75762315295], [253795.7488836188, 172065.88328983408], [257259.0603835971, 168730.84258615127], [259054.851531734, 163728.281530627], [258285.226753961, 157314.74171585232], [257515.60197618802, 152440.45145662356], [255334.99843916463, 147309.61960480385], [253154.3949021412, 142691.87093816607], [249819.35419845834, 140126.4550122562], [252384.77012436822, 138202.3930678238], [257130.7895873015, 142435.3293455751], [259952.74710580238, 145129.01606778044], [262518.16303171223, 146283.45323443992], [266109.745327986, 149105.41075294075], [267777.26567982754, 153979.70101216948], [269444.78603166895, 160393.24082694418], [269829.59842055547, 165780.6142713549], [268803.43205019156, 170654.90453058365], [263287.78780948534, 175529.1947898124], [257002.51879100615, 179377.31867867723], [248536.64623550358, 181429.6514194051], [239557.69049481905, 181429.6514194051], [232246.2551059759, 180660.02664163214], [225704.44449490574, 179633.8602712682], [219675.71706901753, 178992.5062897907], [214673.1560134933, 177838.06912313128], [211851.19849499242, 175914.00717869887], [212107.74008758337, 172835.50806760704], [217623.3843282896, 170911.44612317462], [222112.8621986319, 169115.65497503773], [224678.2781245417, 168730.84258615127], [224421.7365319508, 170141.82134540167]]]


In [ ]:
from shapely import geometry

keep = np.zeros(len(X),dtype=bool)
points = [geometry.Point(Point_X, Point_Y) for Point_X, Point_Y in X]
for mask_points in points_all:
    line = geometry.Polygon(geometry.LineString(mask_points))
    keep |= np.array([line.contains(point) for point in points])

In [ ]:
adata_ctx=adata[keep]

In [ ]:
adata.obs['ctx'] = adata.obs_names.isin(adata_ctx.obs_names)

In [ ]:
adata.obs['ctx'].value_counts()

In [ ]:
adata_no_ctx=adata[adata.obs['ctx']==False]

In [ ]:
X = np.array(adata_no_ctx.obsm['X_multi_spatial'])
res1_values = adata_no_ctx.obs['region_main_1'].values

cluster_colors = {
    'HPC': [1, 0, 0, 1],    # Red
    'CC': [0, 1, 0, 1],     # Green
    'VLMC': [0, 0, 1, 1],   # Blue
    'L2/3': [1, 1, 0, 1]    # Yellow
}

colors = np.zeros((X.shape[0], 4))  

for cluster, color in cluster_colors.items():
    colors[res1_values == cluster] = color

default_color = [0.5, 0.5, 0.5, 1]  
colors[~np.isin(res1_values, list(cluster_colors.keys()))] = default_color

scale_factor = 0.108 

viewer = napari.Viewer()
points_layer = viewer.add_points(
    X,
    size=30,
    face_color=colors,
    scale=[scale_factor, scale_factor],
    metadata={'unit': 'µm'}
)

viewer.scale_bar.visible = True
viewer.scale_bar.unit = 'µm'
viewer.scale_bar.position = 'bottom_right'
viewer.scale_bar.color = 'white'
viewer.scale_bar.font_size = 10

def adjust_point_size(event):
    zoom_level = event.value
    base_size = 6  
    new_size = base_size / zoom_level
    points_layer.size = new_size

viewer.camera.events.zoom.connect(adjust_point_size)

napari.run()


In [ ]:
points = viewer.layers[-1].data
print([[list(p) for p in lines]for lines in points])

In [ ]:
X = np.array(adata_no_ctx.obsm['X_multi_spatial'])

In [ ]:
points_all=[[[51930.55354039352, 8726.255460416261], [49989.27695080358, 9823.498750184492], [47879.1937012493, 11005.14536993489], [45937.91711165935, 13115.228619489177], [44081.04385205158, 15562.92518897214], [42392.977252408156, 18939.058388259], [41042.523972693416, 22737.208237456707], [40536.10399280039, 24509.678167082307], [39692.07069297867, 26788.56807660094], [39016.8440531213, 29827.087955959105], [39016.8440531213, 32359.187855424243], [40367.297332836046, 34553.674434960696], [40958.12064271124, 35397.70773478241], [39185.65071308564, 36663.75768451498], [38172.810753299585, 36663.75768451498], [37835.1974333709, 38689.4376040871], [40704.91065276473, 41221.53750355224], [42899.397232301184, 45272.89734269646], [43659.02720214072, 49408.66051182286], [42730.59057233684, 51012.323781484105], [38763.63406317479, 49324.25718184068], [34037.04758417319, 46623.3506224112], [30070.091075011132, 44935.28402276777], [27200.37785561731, 44175.654052928236], [24921.48794609868, 43247.21742312435], [22136.178056687026, 41559.15082348092], [19519.674827239713, 39533.47090390881], [16649.961607845882, 36410.547694568464], [14708.685018255941, 33118.81782526378], [13864.651718434228, 28729.844666190875], [13864.651718434228, 25860.131446797048], [14624.281688273772, 22315.191587545854], [15805.928308024171, 19192.26837820551], [18422.431537471482, 16153.748498847344], [21376.548086847484, 13621.648599382204], [23655.43799636611, 12102.388659703121], [27369.18451558165, 11764.775339774436], [31758.157674654558, 11427.162019845742], [38594.827403210445, 10076.708740131002], [40620.507322782556, 8641.852130434094], [42477.380582390324, 6531.768880879807], [44081.04385205158, 5603.332251075923], [45937.91711165935, 6025.348900986779]], [[53027.797152134895, 93382.79800831937], [54547.05709181398, 90513.08478892555], [55391.0903916357, 87390.1615795852], [55475.493721617866, 84182.8350402627], [53112.20048211707, 80722.29851099366], [50411.29392268758, 78696.61859142155], [46359.934083543354, 76248.92202193858], [41717.75093452393, 74476.45209231297], [37919.60108532622, 73210.40214258041], [35049.88786593239, 72535.17550272304], [32602.191296449422, 71691.14220290132], [28804.04144725171, 70003.0756032579], [26356.344877768737, 69665.46228332921], [24246.261628214455, 69327.84896340052], [21882.968388713653, 69159.04230343617], [20616.918438981083, 68652.62232354315], [19435.271819230686, 69749.86561331138], [18253.625199480288, 69412.2522933827], [19013.25516931983, 71353.52888297263], [20363.70844903457, 73716.82212247343], [21798.56505873149, 75658.09871206338], [19688.4818091772, 77092.95532176028], [19519.675149212857, 78696.61859142155], [22895.80834849971, 80891.105170958], [26271.94154778657, 82916.78509053012], [28804.04144725171, 84942.46501010223], [30745.318036841647, 86714.93493972783], [33024.20794636028, 87474.56490956737], [34965.48453595022, 89331.43816917513], [35640.71117580759, 91779.1347386581], [36822.35779555799, 94226.83130814109], [37413.18110543319, 97096.5445275349], [39270.05436504097, 96252.5112277132], [41548.94427455959, 95914.89790778451], [43321.41420418519, 96083.70456774885], [45515.90078372164, 95070.8646079628], [47457.177373311584, 94648.84795805193], [49482.857292883695, 94733.25128803411], [51761.74720240233, 94564.44462806977]], [[30154.494726966455, 131870.7190559747], [29816.88140703777, 135331.25558524372], [28550.831457305198, 136428.49887501195], [26778.361527679597, 137441.33883479802], [26018.731557840056, 139973.43873426315], [26356.34487776874, 142674.34529369263], [26862.76485766177, 145628.46184306865], [27537.99149751914, 149004.5950423555], [28213.21813737651, 151621.0982718028], [29901.28473701994, 154912.8281411075], [31842.56132660988, 156854.1047306974], [33361.821266288964, 158204.55801041215], [33193.01460632462, 159470.60796014473], [28213.21813737651, 158457.7680003587], [24161.858298232288, 158120.15468043], [20279.305119052402, 156685.2980707331], [17325.1885696764, 154997.23147108968], [14624.28201024692, 152211.921581678], [11332.552140942238, 149257.805032302], [9053.66223142361, 145544.05851308646], [8040.822271637553, 141661.5053339066], [8040.822271637553, 138369.7754646019], [8378.435591566238, 135078.04559529724], [10150.905521191837, 131786.31572599255], [13864.652040407378, 129507.4258164739], [16227.945279908174, 128241.37586674133], [19435.27181923069, 126722.11592706224], [21798.56505873149, 125709.2759672762], [24668.278278125315, 125624.87263729403], [27200.378177590454, 126300.0992771514], [30238.898056948627, 126806.51925704442], [32602.191296449426, 126468.90593711575], [34290.25789609285, 128494.58585668786], [33361.821266288964, 131701.9123960104], [30745.318036841654, 132039.52571593906]], [[110032.3472504247, 147261.60955795884], [108749.63928746976, 143028.6732802076], [107595.20212081033, 140719.7989468887], [104644.97380601396, 137897.84142838785], [101566.47469492213, 135332.42550247797], [98103.1631949438, 133793.17594693205], [94383.3101023745, 131997.38479879513], [91048.26939869166, 131099.48922472668], [89637.29063944124, 130458.13524324923], [87584.95789871333, 129560.23966918077], [85789.16675057643, 129945.05205806725], [83865.10480614402, 130586.40603954472], [83993.37560243951, 133921.44674322754], [85019.54197280345, 135588.96709506895], [86558.7915283494, 136615.13346543288], [84249.91719503049, 137128.21665061486], [82582.3968431891, 137513.02903950136], [81427.95967652964, 138667.4662061608], [82582.3968431891, 140591.5281505932], [83480.29241725753, 142387.31929873012], [82325.8552505981, 145465.81840982195], [81684.50126912062, 148544.31752091378], [80401.7933061657, 150724.92105793717], [80016.98091727922, 152905.52459496056], [78092.91897284682, 155727.48211346142], [76168.8570284144, 159319.06440973523], [74501.33667657299, 160601.77237269017], [75014.41986175497, 162397.56352082707], [78606.00215802879, 162397.56352082707], [78734.27295432427, 162269.2927245316], [81684.50126912062, 162269.2927245316], [84891.27117650797, 163936.813076373], [87969.77028759981, 165604.33342821442], [90919.99860239617, 165347.79183562344], [95537.74726903393, 163552.00068748652], [99257.60036160325, 161114.85555787213], [102720.91186158157, 158805.98122455325], [105158.05699119595, 156240.5652986434], [107082.11893562836, 153162.06618755157], [108877.91008376527, 151109.7334468237]], [[110288.88884301568, 76969.21318802841], [109006.18088006075, 79406.3583176428], [106184.22336155988, 80432.52468800674], [104003.61982453651, 82484.85742873463], [102079.5578801041, 84665.46096575803], [100155.49593567169, 86717.79370648593], [99257.60036160325, 88770.12644721381], [98872.78797271676, 91079.0007805327], [100283.7667319672, 94414.04148421553], [102079.5578801041, 97620.81139160287], [102079.5578801041, 100314.49811380824], [99898.9543430807, 101597.20607676318], [95152.93488014746, 100827.58129899022], [89380.74904685025, 100827.58129899022], [88226.3118801908, 100442.76891010374], [85917.43754687192, 98262.16537308034], [83608.56321355305, 96466.37422494343], [82069.31365800713, 93259.60431755609], [81427.95967652964, 89668.02202128228], [81556.23047282513, 85819.89813241747], [83480.29241725753, 81202.1494657797], [86045.70834316741, 77482.2963732104], [88611.12426907728, 74532.06805841404], [92972.33134312407, 72479.73531768614], [96435.64284310238, 71581.83974361769], [100668.57912085365, 70812.21496584473], [104260.16141712749, 69144.69461400331], [107210.38973192383, 67861.98665104837], [109904.0764541292, 67733.71585475288], [111186.78441708414, 67990.25744734387], [110545.43043560667, 73249.3600954591], [110545.43043560667, 75686.50522507349]], [[81941.04286171161, 48621.367206724346], [81171.41808393865, 45158.05570674602], [83993.37560243951, 42464.368984540655], [86045.70834316741, 40668.57783640375], [87456.68710241784, 40283.76544751727], [87713.22869500883, 38744.51589197135], [85917.43754687192, 36179.099966061476], [84121.64639873501, 34511.57961422005], [85917.43754687192, 31433.08050312821], [87200.14550982685, 28098.03979944538], [86045.70834316741, 26173.977855012978], [84378.187991326, 23095.478743921132], [82069.31365800713, 20016.979632829287], [80145.2517135747, 15655.772558782508], [79632.16852839274, 12833.815040281652], [75912.31543582342, 11294.56548473573], [80016.98091727922, 8087.795577348391], [82197.5844543026, 8216.066373643884], [84634.72958391698, 7318.170799575431], [87328.41630612235, 9627.045132894313], [90663.45700980519, 11551.107077326715], [93998.49771348802, 13859.981410645598], [97846.62160235281, 13859.981410645598], [101181.66230603565, 13090.35663287264], [105029.78619490046, 16168.855743964481], [108877.91008376527, 19888.70883653379], [112212.95078744809, 23223.749540216624], [113495.65875040302, 25019.540688353536], [113880.47113928951, 29893.830947582286], [112212.95078744809, 33998.49642903808], [107851.74371340132, 37461.8079290164], [101181.66230603565, 41566.4734104722], [96692.18443569337, 44260.16013267757], [92587.51895423757, 46312.49287340546], [88995.93665796376, 47595.2008363604], [84506.45878762148, 48749.63800301984]], [[168010.74717598775, 36307.37027304272], [169678.2675278292, 33870.22514342833], [170832.70469448864, 30150.372050859023], [172500.22504633004, 26430.51895828971], [174039.47460187596, 23352.019847197866], [175322.1825648309, 22325.85347683392], [177374.5153055588, 20530.062328697008], [180068.20202776417, 20145.24993981053], [182120.53476849204, 25276.08179163027], [181863.99317590107, 29252.47647679057], [176604.8905277858, 33741.95434713284], [172756.76663892102, 37076.995050815676], [168267.28876857876, 40027.22336561203], [164803.97726860046, 42207.82690263541], [161468.9365649176, 45158.05521743176], [158518.70825012127, 47338.65875445516], [155568.4799353249, 47723.47114334164], [152874.79321311953, 46569.03397668219], [154670.58436125648, 43747.07645818134], [158390.43745382575, 41053.38973597597], [162238.56134269058, 37461.807439702156], [163008.1861204635, 36563.9118656337], [163649.540101941, 36179.09947674722]], [[182761.88874996954, 107754.2038096326], [177502.7861018543, 105958.41266149569], [172756.76663892102, 102495.10116151736], [167625.9347871013, 99288.33125413003], [161853.74895380408, 96594.64453192467], [160442.77019455365, 95568.47816156072], [158518.70825012127, 92874.79143935535], [156722.91710198435, 89539.75073567253], [155183.66754643843, 86846.06401346716], [154029.23037977898, 82997.94012460236], [153131.3348057105, 79791.17021721501], [153259.60560200602, 77867.10827278261], [153387.8763983015, 75301.69234687275], [151335.5436575736, 73249.35960614485], [149796.2941020277, 70299.13129134849], [151592.0852501646, 66579.27819877918], [154157.50117607447, 64398.67466175579], [157620.8126760528, 63500.77908768733], [159416.6038241897, 63885.59147657381], [161725.47815750857, 65168.29943952876], [163649.540101941, 67348.90297655214], [159031.79143530322, 67733.71536543862], [157236.00028716633, 67348.90297655214], [155825.0215279159, 70170.860495053], [156081.56312050688, 72351.4640320764], [157107.7294908708, 75045.15075428176], [159929.68700937167, 77995.3790690781], [162879.91532416805, 80175.9826061015], [166086.68523155537, 80817.33658757898], [169165.18434264723, 79406.35782832853], [171345.78787967062, 80560.79499498798], [175193.9117685354, 81202.14897646545], [172371.95425003456, 84793.73127273926], [172115.41265744355, 89026.66755049056], [175065.6409722399, 94414.04099490127], [177631.05689814978, 99031.78966153905], [179426.8480462867, 100955.85160597145], [180709.5560092416, 102751.64275410835]], [[187507.9082129028, 164193.35417964976], [185968.6586573569, 165347.79134630918], [185840.38786106138, 168682.83204999202], [186481.74184253885, 172146.14354997035], [182377.07636108305, 172274.41434626587], [176091.80734260386, 170093.81080924245], [172115.41265744355, 168682.83204999202], [168267.28876857876, 167271.8532907416], [162751.64452787256, 165989.14532778668], [159416.6038241897, 163423.7294018768], [155568.4799353249, 158805.98073523902], [152746.52241682404, 155342.66923526072], [151335.5436575736, 149955.29579084995], [152361.71002793754, 148287.77543900858], [154798.85515755194, 147646.42145753108], [157236.00028716633, 146876.79667975812], [159031.79143530322, 144824.46393903025], [160827.58258344015, 144183.10995755275], [163264.72771305454, 143670.02677237077], [165701.87284266888, 143798.29756866628], [169934.8091204202, 147133.33827234912], [171089.24628707962, 152264.17012416883], [173526.39141669398, 156368.83560562466], [176989.70291667234, 156753.64799451112], [179042.0356574002, 158036.35595746606], [181863.99317590107, 160088.68869819396], [185327.3046758794, 161243.1258648534], [187636.17900919827, 163167.18780928582]], [[258441.65073528248, 47466.92955075065], [251258.48614273485, 47851.741939637126], [247025.5498649836, 48364.82512481909], [240098.9268650269, 46953.84636556867], [235994.26138357114, 45799.40919890923], [234070.19943913873, 44388.4304396588], [231504.7835132289, 42849.18088411288], [228682.82599472805, 40155.49416190751], [227913.2012169551, 37461.80743970215], [228939.36758731902, 35281.20390267876], [230606.88793916043, 34383.3083286103], [231119.97112434238, 38616.24460636159], [232274.40829100186, 40668.577347089486], [234839.82421691171, 42720.910087817385], [237790.05253170803, 44003.61805077232], [241125.0932353909, 43747.076458181335], [242279.53040205035, 42849.18088411288], [240740.2808465044, 40155.49416190751], [239457.5728835495, 38487.9738100661], [236379.07377245763, 36307.37027304271], [235224.6366057982, 35537.745495269744], [236763.88616134413, 33485.412754541845], [238816.21890207203, 35281.20390267876], [242022.98880945938, 37205.26584711116], [245229.75871684667, 39001.05699524807], [246512.46667980164, 41823.01451374892], [249590.96579089348, 42720.910087817385], [252028.11092050787, 42464.3684952264], [254721.79764271318, 44131.888847067814], [257928.5675501005, 46184.22158779571], [259980.90029082843, 47082.11716186417]], [[264855.19055005716, 77610.56668019162], [265624.81532783003, 81586.96136535193], [264983.4613463526, 88898.39675419506], [260493.98347601033, 94927.12418008326], [257287.21356862294, 98646.97727265257], [253054.2772908717, 101853.74718003989], [248436.5286242339, 105317.05868001824], [244331.8631427781, 107369.39142074612], [242151.25960575472, 108908.64097629204], [238046.59412429892, 110447.89053183798], [234583.2826243206, 111602.32769849742], [232274.40829100172, 112500.22327256587], [229837.26316138732, 112885.03566145235], [227913.20121695494, 111858.8692910884], [228682.8259947279, 109421.72416147402], [231889.59590211525, 106343.22505038217], [234070.1994391386, 104547.43390224528], [240355.4684576178, 100699.31001338045], [243433.96756870963, 97492.54010599313], [245742.84190202857, 94670.58258749227], [246255.92508721052, 92233.43745787788], [246640.737476097, 89924.563124559], [246384.19588350604, 87743.95958753562], [245871.11269832405, 85948.1684393987], [244460.13393907362, 82613.12773571588], [241638.1764205728, 77867.10827278261], [240098.92686502688, 75686.50473575921], [238687.94810577645, 72223.1932357809], [240996.82243909535, 71966.6516431899], [243433.96756870972, 72223.1932357809], [245229.7587168466, 69786.0481061665], [247795.1746427565, 68375.06934691609], [250104.0489760754, 68503.34014321158], [253439.0896797582, 70812.21447653045], [254850.06843900864, 74275.5259765088], [254978.33923530413, 77995.3790690781], [255619.6932167816, 80304.253402397], [256902.40117973648, 83126.21092089785], [260237.44188341935, 84152.3772912618], [263059.3994019202, 82997.94012460236], [264085.5657722841, 82228.3153468294], [264598.6489574661, 81202.14897646545], [264085.5657722841, 81715.23216164742], [263444.2117908067, 80175.9826061015]], [[224578.16051327196, 168554.56125369653], [224834.70210586296, 171376.5187721974], [227143.57643918184, 174326.7470869937], [229708.99236509172, 175865.99664253966], [233941.928642843, 176507.35062401713], [237918.32332800332, 176507.35062401713], [242022.9888094591, 175609.45504994868], [245101.48792055095, 175096.37186476667], [250617.13216125718, 172915.7683277433], [253310.81888346252, 171504.78956849285], [255363.1516241904, 169837.26921665148], [256902.40117973636, 166630.49930926412], [256902.40117973636, 163295.45860558128], [256517.5887908499, 157523.27277228408], [256261.0471982589, 156240.56480932914], [252669.46490198508, 155727.48162414719], [250488.8613649617, 157138.46038339759], [248693.0702168248, 156112.29401303365], [247025.5498649834, 154188.23206860127], [245486.30030943744, 152648.98251305535], [241894.71801316363, 152392.44092046432], [240098.9268650267, 153803.41967971477], [238559.6773094808, 156625.3771982156], [237405.24014282133, 159319.06392042097], [234583.2826243205, 160601.7718833759], [232915.7622724791, 160858.31347596692], [230093.8047539782, 162782.37542039933], [228298.01360584132, 164449.8957722407], [225091.24369845394, 167784.93647592355]]]


In [ ]:
from shapely import geometry

keep = np.zeros(len(X),dtype=bool)
points = [geometry.Point(Point_X, Point_Y) for Point_X, Point_Y in X]
for mask_points in points_all:
    line = geometry.Polygon(geometry.LineString(mask_points))
    keep |= np.array([line.contains(point) for point in points])

In [ ]:
adata_hpf=adata_no_ctx[keep]
adata_hpf

In [ ]:
adata.obs['hpf'] = adata.obs_names.isin(adata_hpf.obs_names)

In [ ]:
from scipy.spatial import ConvexHull
adata_hpf.obsm['multi_spatial_microns'] = adata_hpf.obsm['X_multi_spatial'] * 0.108

surface_areas = {}
batch_ids = adata_hpf.obs['batch'].unique()

for brain_id in batch_ids:
    mask = adata_hpf.obs['batch'] == brain_id
    coords = adata_hpf.obsm['multi_spatial_microns'][mask, :]
    if coords.shape[0] >= 3:
        hull = ConvexHull(coords)
        area = hull.volume 
        surface_areas[brain_id] = area
    else:
        print(f"Not enough points to compute convex hull for brain {brain_id}.")
        surface_areas[brain_id] = np.nan

surface_area_df = pd.DataFrame.from_dict(surface_areas, orient='index', columns=['surface_area_um2'])
surface_area_df.index.name = 'brain_id'
surface_area_df.reset_index(inplace=True)
surface_area_df['surface_area_mm2'] = surface_area_df['surface_area_um2'] / 1e6
print(surface_area_df)


In [ ]:
adata_hpf.obsm['multi_spatial_microns'] 

In [ ]:
adata_hpf_mg1=adata_hpf[adata_hpf.obs['class']=='Microglia']
adata_hpf_mg=adata_hpf_mg1[~(adata_hpf_mg1.obs['celltype_5']=='unknwon')]

from scipy.spatial import ConvexHull
from scipy.stats import ttest_ind  

batch_to_genotype = {
    'APP_1': 'APP', 'APP_2': 'APP', 'APP_3': 'APP', 'APP_4': 'APP',
    'E4_1': 'E4', 'E4_2': 'E4',
    'TE4_1': 'TE4', 'TE4_2': 'TE4', 'TE4_3': 'TE4', 'TE4_4': 'TE4',
    'WT_1': 'WT', 'WT_2': 'WT'
}

adata_hpf_mg.obs['geno_type'] = adata_hpf_mg.obs['batch'].map(batch_to_genotype)
mg_counts = adata_hpf_mg.obs.groupby(['batch', 'celltype_5']).size().reset_index(name='mg_count')
mg_counts = mg_counts.merge(
    surface_area_df[['brain_id', 'surface_area_mm2']],
    left_on='batch',
    right_on='brain_id',
    how='left'
)

mg_counts['geno_type'] = mg_counts['batch'].map(batch_to_genotype)
missing_geno = mg_counts[mg_counts['geno_type'].isnull()]['batch'].unique()
if len(missing_geno) > 0:
    print(f"Warning: Missing: {missing_geno}")
else:
    print("All batchesgenotype mappings.")

mg_counts['density_per_mm2'] = mg_counts['mg_count'] / mg_counts['surface_area_mm2']

density_stats = mg_counts.groupby(['geno_type', 'celltype_5']).agg(
    Mean_Density=('density_per_mm2', 'mean'),
    Std_Density=('density_per_mm2', 'std')
).reset_index()

custom_order = ['WT', 'APP', 'E4', 'TE4']
density_stats['geno_type'] = pd.Categorical(density_stats['geno_type'], categories=custom_order, ordered=True)
density_stats = density_stats.sort_values(by=['celltype_5', 'geno_type'])

color_map = {
    'WT': 'skyblue',
    'E4': '#bfef45',
    'APP': '#ec008c',
    'TE4': '#fabed4'
}
density_stats['color'] = density_stats['geno_type'].map(color_map)

class_names = density_stats['celltype_5'].unique()
num_genotypes = len(custom_order)
spacing = 1.5  
bar_width = 0.8

x_positions = []
for i, class_name in enumerate(class_names):
    start_pos = i * (num_genotypes + spacing)
    positions = np.arange(start_pos, start_pos + num_genotypes)
    x_positions.extend(positions)

density_stats['x_pos'] = x_positions

plt.figure(figsize=(12, 6))
plt.bar(
    density_stats['x_pos'], 
    density_stats['Mean_Density'], 
    yerr=density_stats['Std_Density'], 
    color=density_stats['color'], 
    capsize=4, 
    width=bar_width
)

plt.title('HPF Microglia Density per Surface Area by Genotype')
plt.xlabel('Microglia Clusters')
plt.ylabel('Density (cells/mm²)')

class_ticks = []
for i, class_name in enumerate(class_names):
    start = i * (num_genotypes + spacing)
    center = start + (num_genotypes - 1) / 2
    class_ticks.append(center)
plt.xticks(class_ticks, class_names, rotation=0)

legend_elements = [plt.Line2D([0], [0], color=color_map[geno], lw=10) for geno in custom_order]
plt.legend(legend_elements, custom_order, title="Genotype", loc='upper right')
plt.tight_layout()

output_path = os.path.join(output_folder, 'Microglia_Density_by_HPF_1.pdf')
plt.savefig(output_path, format='pdf', dpi=300, bbox_inches='tight')

mg_counts.to_csv(os.path.join(output_folder, 'mg_counts_hpf.csv'), index=False)
density_stats.to_csv(os.path.join(output_folder, 'mg_density_stats_hpf.csv'), index=False)
plt.show()

t_test_results = []
comparison_pairs = {
    'WT_vs_APP': ('WT', 'APP'),
    'E4_vs_TE4': ('E4', 'TE4')
}

for celltype in mg_counts['celltype_5'].unique():
    data = mg_counts[mg_counts['celltype_5'] == celltype]
    p_values = {'celltype_5': celltype}
    for comparison, (geno1, geno2) in comparison_pairs.items():
        group1 = data[data['geno_type'] == geno1]['density_per_mm2']
        group2 = data[data['geno_type'] == geno2]['density_per_mm2']

        if len(group1) > 1 and len(group2) > 1:
            t_stat, p_val = ttest_ind(group1, group2, equal_var=False)
            p_values[f'{comparison}_pvalue'] = p_val
        else:
            p_values[f'{comparison}_pvalue'] = np.nan  
    t_test_results.append(p_values)
t_test_df = pd.DataFrame(t_test_results)
t_test_output_path = os.path.join(output_folder, 'mg_density_t_tests_hpf_1.csv')
t_test_df.to_csv(t_test_output_path, index=False)

print(f"T-test results saved to {t_test_output_path}")


In [ ]:
adata_ctx.obsm['multi_spatial_microns'] = adata_ctx.obsm['X_multi_spatial'] * 0.108

surface_areas = {}
batch_ids = adata_ctx.obs['batch'].unique()
for brain_id in batch_ids:
    mask = adata_ctx.obs['batch'] == brain_id
    coords = adata_ctx.obsm['multi_spatial_microns'][mask, :]
    if coords.shape[0] >= 3:
        hull = ConvexHull(coords)
        area = hull.volume  
        surface_areas[brain_id] = area
    else:
        print(f"Not enough points to compute convex hull for brain {brain_id}.")
        surface_areas[brain_id] = np.nan

surface_area_df = pd.DataFrame.from_dict(surface_areas, orient='index', columns=['surface_area_um2'])
surface_area_df.index.name = 'brain_id'
surface_area_df.reset_index(inplace=True)
surface_area_df['surface_area_mm2'] = surface_area_df['surface_area_um2'] / 1e6
print(surface_area_df)

In [ ]:
adata_ctx_mg1=adata_ctx[adata_ctx.obs['class']=='Microglia']
adata_ctx_mg=adata_ctx_mg1[~(adata_ctx_mg1.obs['celltype_5']=='unknwon')]

batch_to_genotype = {
    'APP_1': 'APP', 'APP_2': 'APP', 'APP_3': 'APP', 'APP_4': 'APP',
    'E4_1': 'E4', 'E4_2': 'E4',
    'TE4_1': 'TE4', 'TE4_2': 'TE4', 'TE4_3': 'TE4', 'TE4_4': 'TE4',
    'WT_1': 'WT', 'WT_2': 'WT'
}
adata_ctx_mg.obs['geno_type'] = adata_ctx_mg.obs['batch'].map(batch_to_genotype)
mg_counts = adata_ctx_mg.obs.groupby(['batch', 'celltype_5']).size().reset_index(name='mg_count')
mg_counts = mg_counts.merge(
    surface_area_df[['brain_id', 'surface_area_mm2']],
    left_on='batch',
    right_on='brain_id',
    how='left'
)

mg_counts['geno_type'] = mg_counts['batch'].map(batch_to_genotype)
missing_geno = mg_counts[mg_counts['geno_type'].isnull()]['batch'].unique()
if len(missing_geno) > 0:
    print(f"Warning: Missing genotype : {missing_geno}")
else:
    print("All batches genotype mappings.")
mg_counts['density_per_mm2'] = mg_counts['mg_count'] / mg_counts['surface_area_mm2']
density_stats = mg_counts.groupby(['geno_type', 'celltype_5']).agg(
    Mean_Density=('density_per_mm2', 'mean'),
    Std_Density=('density_per_mm2', 'std')
).reset_index()

custom_order = ['WT', 'APP', 'E4', 'TE4']
density_stats['geno_type'] = pd.Categorical(density_stats['geno_type'], categories=custom_order, ordered=True)
density_stats = density_stats.sort_values(by=['celltype_5', 'geno_type'])

color_map = {
    'WT': 'skyblue',
    'E4': '#bfef45',
    'APP': '#ec008c',
    'TE4': '#fabed4'
}
density_stats['color'] = density_stats['geno_type'].map(color_map)

class_names = density_stats['celltype_5'].unique()
num_genotypes = len(custom_order)
spacing = 1.5  
bar_width = 0.8
x_positions = []
for i, class_name in enumerate(class_names):
    start_pos = i * (num_genotypes + spacing)
    positions = np.arange(start_pos, start_pos + num_genotypes)
    x_positions.extend(positions)

density_stats['x_pos'] = x_positions
plt.figure(figsize=(12, 6))
plt.bar(
    density_stats['x_pos'], 
    density_stats['Mean_Density'], 
    yerr=density_stats['Std_Density'], 
    color=density_stats['color'], 
    capsize=4, 
    width=bar_width
)

plt.title('Isocortex Microglia Density per Surface Area by Genotype')
plt.xlabel('Microglia Clusters')
plt.ylabel('Density (cells/mm²)')

class_ticks = []
for i, class_name in enumerate(class_names):
    start = i * (num_genotypes + spacing)
    center = start + (num_genotypes - 1) / 2
    class_ticks.append(center)

plt.xticks(class_ticks, class_names, rotation=0)

legend_elements = [plt.Line2D([0], [0], color=color_map[geno], lw=10) for geno in custom_order]
plt.legend(legend_elements, custom_order, title="Genotype", loc='upper right')

plt.tight_layout()

output_path = os.path.join(output_folder, 'Microglia_Density_by_ctx_1.pdf')
plt.savefig(output_path, format='pdf', dpi=300, bbox_inches='tight')

mg_counts.to_csv(os.path.join(output_folder, 'mg_counts_ctx.csv'), index=False)
density_stats.to_csv(os.path.join(output_folder, 'mg_density_stats_ctx.csv'), index=False)
plt.show()

t_test_results = []
comparison_pairs = {
    'WT_vs_APP': ('WT', 'APP'),
    'E4_vs_TE4': ('E4', 'TE4')
}
for celltype in mg_counts['celltype_5'].unique():
    data = mg_counts[mg_counts['celltype_5'] == celltype]
    p_values = {'celltype_5': celltype}
    for comparison, (geno1, geno2) in comparison_pairs.items():
        group1 = data[data['geno_type'] == geno1]['density_per_mm2']
        group2 = data[data['geno_type'] == geno2]['density_per_mm2']
        if len(group1) > 1 and len(group2) > 1:
            t_stat, p_val = ttest_ind(group1, group2, equal_var=False)
            p_values[f'{comparison}_pvalue'] = p_val
        else:
            p_values[f'{comparison}_pvalue'] = np.nan  
    t_test_results.append(p_values)

t_test_df = pd.DataFrame(t_test_results)
t_test_output_path = os.path.join(output_folder, 'mg_density_t_tests_ctx_1.csv')
t_test_df.to_csv(t_test_output_path, index=False)

print(f"T-test results saved to {t_test_output_path}")